# 5 - Creating an Azure ML Pipeline

This notebook will create a pipeline within the Azure Machine Learning service that prepares data, trains a model, and registers the model within a workspace.

This pipeline can sit within a larger pipeline that integrates model deployment as well as continuous delivery for the code that is leveraged.  While that is beyond the scope of this notebook, you can learn more about this overall approach here:

[MLOps-Python Reference Architecture](https://github.com/microsoft/MLOpsPython)

## Imports

First, we will need to update several modules for this notebook:

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore
from azureml.core import Experiment
from azureml.core.compute import AmlCompute
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.dnn import TensorFlow
from azureml.widgets import RunDetails
from azureml.opendatasets import MNIST
print("Azure ML SDK Version: ", azureml.core.VERSION)

## References

We will need to get a reference to both the Azure ML workspace as well as the current experiment:

In [ ]:
# Get a reference to the workspace
ws = Workspace.from_config()
print("Azure ML Workspace")
print(f'Name: {ws.name}')
print(f'Location: {ws.location}')
print(f'Resource Group: {ws.resource_group}')

# Create an experiment, or get a reference to the experiment if it already exists
experiment_name = 'keras-mnist'
exp = Experiment(workspace=ws, name=experiment_name)
print("Azure ML Experiment")
print(f'ID: {exp.id}')
print(f'Name: {exp.name}')

Next, we will need to get a reference to the compute cluster we are leveraging from the workspace:

In [ ]:
# Create a name for our new cluster
cpu_cluster_name = 'tdsp-cluster'

# Verify that cluster does not exist already
try:
    cpu_cluster = AmlCompute(workspace=ws, name=cpu_cluster_name)
    print('Cluster already exists.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',
                                                           max_nodes=4)
    cpu_cluster = AmlCompute.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Pipeline Creation

Now that we have the needed references in place, we will need to create the actual pipeline.

### Imports

First, we will need to import some additional modules that are specific to our pipeline:

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

### Pipeline Package Dependencies

Next, we need to define the `RunConfiguration` that will be used to run the pipeline steps.  In this case, we need to be sure that we include `tensorflow` and `keras` from conda and `azureml-opendatasets` from pip (which provides the MNIST dataset that we have been working with). 

In [ ]:
# DEFINE our conda and pip dependencies for the pipeline environment
conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow','keras'])
conda_dependencies.add_pip_package('azureml-opendatasets')
run_config = RunConfiguration(conda_dependencies=conda_dependencies)

### Data Storage and Inputs/Outputs

Next, we need to define where we will store the outputs of each pipeline step.  In this case, we will be storing this in blob storage.  In addition, we need to define the two `PipelineData` instances: the prepared data and the compiled model.

We will be leveraging the `upload` mode on both of these `PipelineData` instances, which means that they will be uploaded after out step completes to the datastore.  These will then be made available to subsequent steps that require them as inputs.

In [ ]:
# Blob storage associated with the workspace
blob_store = Datastore(ws, "workspaceblobstore")

# Create our Pipeline Data references
mnist_data = PipelineData("mnist_data",
                          datastore=blob_store,
                          output_mode="upload",
                          output_path_on_compute="data/mnist.npy",
                          output_overwrite=True)

model_data = PipelineData("mnist_model",
                          datastore=blob_store,
                          output_mode="upload",
                          output_path_on_compute="outputs/mnist.h5",
                          output_overwrite=True)

### Pipeline Steps

Next, we are ready to define our pipeline steps.  In this case, we chose to make each step a `PythonScriptStep`. We could have chosen to use the `EstimatorStep` for training, but for simplicity we will not use it in this initial pipeline. 

In [ ]:
# Directory where scripts reside
scripts_directory = './pipeline'

# Create Pipeline Steps
prep_data_step = PythonScriptStep(name="Prep Data",
                                  script_name="prepData.py",
                                  compute_target=cpu_cluster,
                                  outputs=[mnist_data],
                                  source_directory=scripts_directory,
                                  runconfig=run_config)

train_step = PythonScriptStep(name="Train Model",
                              script_name="train.py",
                              arguments=["--input-data", mnist_data],
                              compute_target=cpu_cluster,
                              inputs=[mnist_data],
                              outputs=[model_data],
                              source_directory=scripts_directory,
                              runconfig=run_config)

register_step = PythonScriptStep(name="Register Model",
                                 script_name="register.py",
                                 arguments=["--input-data", model_data],
                                 compute_target=cpu_cluster,
                                 inputs=[model_data],
                                 source_directory=scripts_directory,
                                 runconfig=run_config)

pipeline_steps = [prep_data_step, train_step, register_step]

## Create and Run the Pipeline

Now that we have created the pipeline steps, we will now create the instance of the pipeline:

In [ ]:
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print ("Pipeline is built")

### Validation

Before we execute the pipeline, we can validate the configuration by calling the `validate` method on our pipeline instance:

In [ ]:
pipeline.validate()
print("Pipeline validation complete")

### Execution

Since we have a validated pipeline instance, we can now execute the pipeline.  We can utilize the same `RunDetails` utility to track the progress of the pipeline run directly from our notebook:

In [ ]:
pipeline_run = exp.submit(pipeline)
RunDetails(pipeline_run).show()